## Import dependent libraries

In [18]:
from pandas import read_csv
from matplotlib import pyplot
import math
from numpy import concatenate, array, asarray
from pandas import DataFrame
from pandas import concat
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Model, model_from_json
from keras.layers import Input
from keras.layers import Dense, Dropout, BatchNormalization
from keras.layers.recurrent import LSTM
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.layers.core import Activation
from keras.callbacks import ReduceLROnPlateau
from keras.utils import plot_model
import random

## Convert series to supervised learning

In [3]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## Encode and Scale the values

In [237]:
dataset = read_csv('music_data.csv', header=0)
print(dataset[0:10])
values = dataset.values
print(values[0:10])
# integer encode direction
# encoder = LabelEncoder()
# values[:, 0] = encoder.fit_transform(values[:, 0])
# print(values[0:10])
# ensure all data is float
values = values.astype('float32')
print(values[0:10])
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
print(scaled[0:10])
# frame as supervised learning

   59  84897
0  60  25456
1  59  14880
2  62  25820
3  60  21361
4  59  21153
5  60  26320
6  58  20404
7  57  86203
8  59  20645
9  57  19400
[[   60 25456]
 [   59 14880]
 [   62 25820]
 [   60 21361]
 [   59 21153]
 [   60 26320]
 [   58 20404]
 [   57 86203]
 [   59 20645]
 [   57 19400]]
[[  6.00000000e+01   2.54560000e+04]
 [  5.90000000e+01   1.48800000e+04]
 [  6.20000000e+01   2.58200000e+04]
 [  6.00000000e+01   2.13610000e+04]
 [  5.90000000e+01   2.11530000e+04]
 [  6.00000000e+01   2.63200000e+04]
 [  5.80000000e+01   2.04040000e+04]
 [  5.70000000e+01   8.62030000e+04]
 [  5.90000000e+01   2.06450000e+04]
 [  5.70000000e+01   1.94000000e+04]]
[[ 0.59999996  0.05136999]
 [ 0.58999997  0.02775489]
 [ 0.62        0.05218277]
 [ 0.59999996  0.04222629]
 [ 0.58999997  0.04176185]
 [ 0.59999996  0.05329921]
 [ 0.57999998  0.04008941]
 [ 0.56999999  0.1870117 ]
 [ 0.58999997  0.04062754]
 [ 0.56999999  0.03784758]]


In [238]:
# specify the number of lag hours
n_hours = 10
n_features = 2
# frame as supervised learning
reframed = series_to_supervised(scaled, n_hours, 1)
print(reframed[0:1:])
print(reframed.shape)


    var1(t-10)  var2(t-10)  var1(t-9)  var2(t-9)  var1(t-8)  var2(t-8)  \
10         0.6     0.05137       0.59   0.027755       0.62   0.052183   

    var1(t-7)  var2(t-7)  var1(t-6)  var2(t-6)    ...     var1(t-4)  \
10        0.6   0.042226       0.59   0.041762    ...          0.58   

    var2(t-4)  var1(t-3)  var2(t-3)  var1(t-2)  var2(t-2)  var1(t-1)  \
10   0.040089       0.57   0.187012       0.59   0.040628       0.57   

    var2(t-1)  var1(t)   var2(t)  
10   0.037848      0.6  0.053277  

[1 rows x 22 columns]
(12236, 22)


In [239]:
values = reframed.values
n_train_hours = 9000
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
n_obs = n_hours * n_features


In [240]:
train_X, train_y_notes = train[:, :n_obs], train[:, -n_features]
train_y_length = train[:, -n_features -1]

test_X, test_y_notes = test[:, :n_obs], test[:, -n_features]
test_y_length = test[:, -n_features -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))
print(train_X.shape, train_y_notes.shape, test_X.shape, test_y_notes.shape)


(9000, 10, 2) (9000,) (3236, 10, 2) (3236,)


In [396]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [397]:
loaded_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_main (InputLayer)          (None, 10, 2)         0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 10, 128)       67072       input_main[0][0]                 
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 10, 128)       512         lstm_1[0][0]                     
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 10, 128)       0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [422]:
# make a prediction
rand = random.randint(0,3000)
print(rand)
testx = dataset[rand:rand+11]
input_values = testx
# testx = [[60, 25456],
# [59, 14880],
# [62, 25820],
# [60, 21361],
# [59, 21153],
# [60, 26320],
# [58, 20404],
# [57, 86203],
# [59, 20645],
# [57, 19400],
# [60, 26310]]

210


In [423]:
def process_array_for_prediction(testx):
    testx = DataFrame(data = testx)
    testx = testx.values
    testx = testx.astype('float32')
    testx = scaler.transform(testx)
    testx = series_to_supervised(testx, n_hours, 1)
    testx = testx.values
    testx = testx[:, :n_obs]
    testx = testx.reshape((testx.shape[0], n_hours, n_features))
    return testx

In [424]:
testx = process_array_for_prediction(testx)
print(testx)

[[[ 0.63        0.01773142]
  [ 0.63999999  0.00245842]
  [ 0.63        0.00473151]
  [ 0.57999998  0.02807196]
  [ 0.52999997  0.0039299 ]
  [ 0.          0.00496819]
  [ 0.52999997  0.01669759]
  [ 0.53999996  0.02733734]
  [ 0.47        0.00482082]
  [ 0.57999998  0.02918394]]]


In [425]:
prediction = loaded_model.predict(testx)
prediction = asarray(prediction).ravel().reshape(-1,2)

In [426]:
prediction = scaler.inverse_transform(prediction)
prediction = prediction.astype('int')

In [427]:

print("INPUT_VALUES:")
print(input_values.values)
print("PREDICTION:")
print(prediction)

INPUT_VALUES:
[[   63 10391]
 [   64  3551]
 [   63  4569]
 [   58 15022]
 [   53  4210]
 [    0  4675]
 [   53  9928]
 [   54 14693]
 [   47  4609]
 [   58 15520]
 [   61  9479]]
PREDICTION:
[[   55 12378]]
